# 《动手学深度学习》-Pytorch-笔记  
https://tangshusen.me/Dive-into-DL-PyTorch/#/chapter01_DL-intro/deep-learning-intro

## 1.创建tensor

In [1]:
import torch
x = torch.ones(5,3)
print(x)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])


In [4]:
# 0到1之间的均匀分布随机数
y = torch.rand(5, 3, dtype=torch.float)
print(y)

tensor([[0.3978, 0.9225, 0.2420],
        [0.1921, 0.6477, 0.5308],
        [0.0065, 0.1388, 0.8290],
        [0.1814, 0.4887, 0.7650],
        [0.3092, 0.7973, 0.2034]])


### 指定输出

In [22]:
result = torch.empty(5, 3)
torch.add(x, y , out=result)
print(result)

tensor([[1.0547, 1.6257, 1.7430],
        [1.7060, 1.9304, 1.0602],
        [1.6800, 1.9177, 1.7664],
        [1.8921, 1.7341, 1.3179],
        [1.8357, 1.2088, 1.8400]])


### 对tensor进行数据索引，索引出来的的结果与原数据共享内存，也即修改一个，另一个会跟着修改。

In [25]:
xx = result[0, :]
xx += 1
print(xx)
print(result)

tensor([2.0547, 2.6257, 2.7430])
tensor([[2.0547, 2.6257, 2.7430],
        [1.7060, 1.9304, 1.0602],
        [1.6800, 1.9177, 1.7664],
        [1.8921, 1.7341, 1.3179],
        [1.8357, 1.2088, 1.8400]])


### 选择函数
``index_select(input, dim, index)`` 在指定维度上进行选取，比如选取某些行，某些列。  
``mask_select(input, mask)``  
``nonzero(input)``                    非0元素的下标  
``gather(input, dim, index)``         根据index，在dim维度上选取数据，输出的size与index一致。

### 改变形状
使用view()来改变Tensor的形状  
注：view()返回的新Tensor与源Tensor虽然有不同的size，但是是共享内存的。

### 返回一个新的副本（不共享内存）
推荐先用``clone``创建一个副本，再使用``view``。  
使用``clone``还有一个好处是会被记录在计算图中，即梯度回传到副本时也会传到源``Tensor``。

### 运行时的内存开销
``y = x + y`` 这样的运算会开辟新内存。  
不会开辟新内存的运算方式可写为如下：  
``y\[:\] = y + x``  
``torch.add(x, y, out=y)``  
``y += x``  
``y.add_(x)``

### tensor和numpy相互转换
a = torch.ones(5)  
b = a.numpy() 
   
a = np.ones(5)  
b = torch.from_numpy(a)

常用方法：  
``c = torch.tensor(a)``  
注： 这种方法会对数据进行拷贝，即返回的``Tensor``与原数据不共享内存。

### Tensor on GPU
使用方法``to()``将Tensor在CPU和GPU上相互移动

In [33]:
x = torch.ones(3,3)
if torch.cuda.is_available():
    device = torch.device("cuda")
    y = torch.ones_like(x, device=device) # 直接创建一个在GPU上的tensor
    x = x.to(device)
    z = x + y
    print(z)
    print(z.to("cpu", torch.double)) # to()还可以同时改变数据类型

tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')
tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]], dtype=torch.float64)


### 自动求梯度
Tensor是一个类，将其属性``.requires_grad``设置为True，则可对其进行track，进行反向传播。  
``.detach()``将其从追踪记录中分离出来。  
``with torch.no_grad()``将不想被追踪的操作代码块包裹起来，常用在评估模型的时候。  
`` y.backward(w)``求的不是 y 对 x 的导数，而是 l = torch.sum(y*w) 对 x 的导数。w 可以视为 y 的各分量的权重，也可以视为遥远的损失函数 l 对 y 的偏导数。若 y 为标量，w 取默认值 1.0，才是按照我们通常理解的那样，求 y 对 x 的导数。  
注意：grad在反向传播过程中是累加的(accumulated)，这意味着每一次运行反向传播，梯度都会累加之前的梯度，所以一般在反向传播之前需把``梯度清零``。